### Objective : In this notebook we will play around with the spotify datasets and do the following things
                
    1. Read the pickle file of summarised datasets
    2. Train a word 2 vec model using skip gram with window size as a hyperparametrs
    3. Play around with the vectors received from this excercise 
    4. Try creating two function which return most similar songs to particular songs
    5. Take 3 songs as list and return a playlist of 10 words

In [16]:
### Load the required packages in the required format
import pandas as pd
import os
import warnings
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
plt.style.use('ggplot')

In [17]:
### Load the pickled datasets 
with open('spotify_summary.pickle','rb') as dataset:
    spotify_summary = pickle.load(dataset)
    print (" The dataset is loaded succesfully")
    print (" The shape of the dataset is as follows",spotify_summary.shape)
    print (spotify_summary.head(5))

 The dataset is loaded succesfully
 The shape of the dataset is as follows (229180, 3)
                            user_id      playlistname  \
0  00055176fea33f6e027cd3302289378b              favs   
1  0007f3dd09c91198371454c608d47f22              2014   
2  0007f3dd09c91198371454c608d47f22         Fav songs   
3  0007f3dd09c91198371454c608d47f22         Sad songs   
4  000b0f32b5739f052b9d40fcc5c41079  Agnetha Fältskog   

                                           trackname  
0  [9619, 2591, 46683, 9620, 1138379, 37346, 6335...  
1                [174985, 1541, 878603, 17550, 5303]  
2  [1854415, 174985, 1684382, 955407, 19605, 1482...  
3                                 [1510871, 1448429]  
4                                  [1281658, 487582]  


In [18]:
### Gensim takes input as a list of list. Our tracknames are already a list convert them to list of list
spotify_wrd2vec_input = [ x for x in spotify_summary['trackname']]
print ("Input data is ready for gensim models")
print ("The number of input playlists we have are as follows :",len(spotify_wrd2vec_input))

Input data is ready for gensim models
The number of input playlists we have are as follows : 229180


In [19]:
### Tip : Always read the help before using any function so we will have a look at the word2vec functions
help(gensim.models.Word2Vec())

Help on Word2Vec in module gensim.models.word2vec object:

class Word2Vec(gensim.models.base_any2vec.BaseWordEmbeddingsModel)
 |  Word2Vec(sentences=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=())
 |  
 |  Class for training, using and evaluating neural networks described in https://code.google.com/p/word2vec/
 |  
 |  If you're finished training a model (=no more updates, only querying)
 |  then switch to the :mod:`gensim.models.KeyedVectors` instance in wv
 |  
 |  The model can be stored/loaded via its :meth:`~gensim.models.word2vec.Word2Vec.save()` and
 |  :meth:`~gensim.models.word2vec.Word2Vec.load()` methods, or stored/loaded in a format
 |  compatible with the original word2vec implementation via `wv.save_word2vec_format()`
 |  a

In [20]:
### Define traing the word 2 vec model we will use Skip Gram using negative sampling as oftmax can be slow
# seed = 1000, hs = 0,negative = 10,workers=10,iter = 100)
### Skip Gram : Predict Context given the middle word works well with infrequent datasets. Good idea for songs as some songs may ne liked by a few users oly
model = gensim.models.Word2Vec(spotify_wrd2vec_input, size = 200 , window = 4 , min_count = 15,
                               seed = 1000, hs = 0,negative = 10,workers=16,iter = 100)
                              

2020-05-03 17:05:07,687 : INFO : collecting all words and their counts
2020-05-03 17:05:07,690 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-03 17:05:07,917 : INFO : PROGRESS: at sentence #10000, processed 539520 words, keeping 236969 word types
2020-05-03 17:05:08,122 : INFO : PROGRESS: at sentence #20000, processed 1048854 words, keeping 375470 word types
2020-05-03 17:05:08,317 : INFO : PROGRESS: at sentence #30000, processed 1587779 words, keeping 500059 word types
2020-05-03 17:05:08,507 : INFO : PROGRESS: at sentence #40000, processed 2120525 words, keeping 595302 word types
2020-05-03 17:05:08,772 : INFO : PROGRESS: at sentence #50000, processed 2684062 words, keeping 699687 word types
2020-05-03 17:05:08,940 : INFO : PROGRESS: at sentence #60000, processed 3142250 words, keeping 775550 word types
2020-05-03 17:05:09,235 : INFO : PROGRESS: at sentence #70000, processed 3919570 words, keeping 935067 word types
2020-05-03 17:05:09,407 : INFO : 

2020-05-03 17:05:49,548 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-03 17:05:49,568 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-03 17:05:49,571 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-03 17:05:49,574 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-03 17:05:49,593 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-03 17:05:49,594 : INFO : EPOCH - 1 : training on 12601573 raw words (8430157 effective words) took 32.3s, 261309 effective words/s
2020-05-03 17:05:50,619 : INFO : EPOCH 2 - PROGRESS: at 3.58% examples, 294332 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:05:51,656 : INFO : EPOCH 2 - PROGRESS: at 7.27% examples, 295374 words/s, in_qsize 32, out_qsize 0
2020-05-03 17:05:52,678 : INFO : EPOCH 2 - PROGRESS: at 10.33% examples, 277775 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:05:53,678 : INFO : EPOCH 2 - PROGRESS: at 14.47% exa

2020-05-03 17:06:43,183 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-03 17:06:43,186 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-03 17:06:43,220 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-03 17:06:43,225 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-03 17:06:43,285 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-03 17:06:43,288 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-03 17:06:43,289 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-05-03 17:06:43,291 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-05-03 17:06:43,292 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-03 17:06:43,296 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-03 17:06:43,328 : INFO : worker thread finished; awaiting finish of 2 more thre

2020-05-03 17:07:34,706 : INFO : EPOCH 5 - PROGRESS: at 77.86% examples, 260443 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:07:35,761 : INFO : EPOCH 5 - PROGRESS: at 81.00% examples, 259744 words/s, in_qsize 30, out_qsize 2
2020-05-03 17:07:36,788 : INFO : EPOCH 5 - PROGRESS: at 84.48% examples, 261036 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:07:37,811 : INFO : EPOCH 5 - PROGRESS: at 87.59% examples, 260114 words/s, in_qsize 31, out_qsize 2
2020-05-03 17:07:38,845 : INFO : EPOCH 5 - PROGRESS: at 91.27% examples, 260655 words/s, in_qsize 31, out_qsize 1
2020-05-03 17:07:39,978 : INFO : EPOCH 5 - PROGRESS: at 94.43% examples, 259487 words/s, in_qsize 29, out_qsize 3
2020-05-03 17:07:41,011 : INFO : EPOCH 5 - PROGRESS: at 97.75% examples, 260413 words/s, in_qsize 27, out_qsize 0
2020-05-03 17:07:41,269 : INFO : worker thread finished; awaiting finish of 15 more threads
2020-05-03 17:07:41,321 : INFO : worker thread finished; awaiting finish of 14 more threads
2020-05-03 17:07:4

2020-05-03 17:08:21,756 : INFO : EPOCH 7 - PROGRESS: at 45.76% examples, 313830 words/s, in_qsize 32, out_qsize 1
2020-05-03 17:08:22,780 : INFO : EPOCH 7 - PROGRESS: at 50.22% examples, 318091 words/s, in_qsize 32, out_qsize 0
2020-05-03 17:08:23,800 : INFO : EPOCH 7 - PROGRESS: at 53.85% examples, 318663 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:08:24,807 : INFO : EPOCH 7 - PROGRESS: at 57.10% examples, 316334 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:08:25,854 : INFO : EPOCH 7 - PROGRESS: at 61.27% examples, 317613 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:08:26,899 : INFO : EPOCH 7 - PROGRESS: at 65.06% examples, 316748 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:08:27,930 : INFO : EPOCH 7 - PROGRESS: at 69.18% examples, 316414 words/s, in_qsize 32, out_qsize 3
2020-05-03 17:08:28,978 : INFO : EPOCH 7 - PROGRESS: at 73.38% examples, 316170 words/s, in_qsize 31, out_qsize 0
2020-05-03 17:08:30,013 : INFO : EPOCH 7 - PROGRESS: at 76.73% examples, 315935 words/s,

...

2020-05-03 17:43:14,187 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-05-03 17:43:14,189 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-03 17:43:14,191 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-03 17:43:14,193 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-03 17:43:14,194 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-03 17:43:14,199 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-03 17:43:14,254 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-03 17:43:14,278 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-05-03 17:43:14,281 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-05-03 17:43:14,287 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-03 17:43:14,332 : INFO : worker thread finished; awaiting finish of 3 more thr

2020-05-03 17:44:01,116 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-05-03 17:44:01,118 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-03 17:44:01,119 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-03 17:44:01,121 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-03 17:44:01,122 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-03 17:44:01,124 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-03 17:44:01,125 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-03 17:44:01,127 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-05-03 17:44:01,171 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-05-03 17:44:01,185 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-03 17:44:01,192 : INFO : worker thread finished; awaiting finish of 3 more thr

2020-05-03 17:44:47,573 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-05-03 17:44:47,576 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-03 17:44:47,579 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-03 17:44:47,626 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-03 17:44:47,681 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-03 17:44:47,685 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-03 17:44:47,721 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-03 17:44:47,724 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-05-03 17:44:47,744 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-05-03 17:44:47,747 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-03 17:44:47,754 : INFO : worker thread finished; awaiting finish of 3 more thr

2020-05-03 17:45:34,463 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-05-03 17:45:34,507 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-03 17:45:34,509 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-03 17:45:34,549 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-03 17:45:34,614 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-03 17:45:34,619 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-03 17:45:34,658 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-03 17:45:34,665 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-05-03 17:45:34,692 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-05-03 17:45:34,698 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-03 17:45:34,703 : INFO : worker thread finished; awaiting finish of 3 more thr

2020-05-03 17:46:21,245 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-05-03 17:46:21,302 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-05-03 17:46:21,334 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-05-03 17:46:21,336 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-05-03 17:46:21,343 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-05-03 17:46:21,386 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-05-03 17:46:21,408 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-05-03 17:46:21,420 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-05-03 17:46:21,477 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-05-03 17:46:21,484 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-05-03 17:46:21,490 : INFO : worker thread finished; awaiting finish of 3 more thr

In [22]:
### Pickle the model datasets and save it to a pickle file 

with open('model_spotify_word2vec.pickle','wb') as model_file:
    pickle.dump(model,model_file)
    print (" Dumping the model succesful ")
    print (" The model is dumped at this location :",os.getcwd())

 Dumping the model succesful 
 The model is dumped at this location : C:\Users\ash\Desktop\NLP-DL


In [24]:
### From the dump load the model dictionary and model pickle files
with open('model_spotify_word2vec.pickle','rb') as model_file:
    model_spotify = pickle.load(model_file)

### Load the pickle files stored for song to numeric 
with open('track_map_dict.pickle','rb') as dict1:
    track_dict= pickle.load( dict1)
print ("Track dict has {} observations".format(len(track_dict)))
#### Load the prcikle file for artist to numeric
with open('track_map_comp_dict.pickle','rb') as dict2:
    track_map_comp_dict = pickle.load(dict2)
print ("Track dict has {} observations".format(len(track_map_comp_dict)))

Track dict has 1866246 observations
Track dict has 1978500 observations


In [29]:
#### Define a function which takes as input songs from list and returns similar songs
def similar_songs(songname):
    ''' Gets the songname from user and return the 5 songs similar'''
    song_id = track_dict[songname]
    print ("Searching for songs similar to :",songname)
    
    similar = model_spotify.most_similar(song_id)
    print ("Similar songs are as follow")
    for i in similar[:]:
        print (track_map_comp_dict[i[0]])
        
    

In [55]:
#### Define a function which takes as input songs from list and returns similar songs
def create_play_list(list_songs):
    ''' Gets the songname from user and return the 5 songs similar'''  
    list1 = []
    for i in list_songs:
        list1.append(track_dict[i])      
        
    print ("Searching for songs similar to :",list_songs)
    
    similar = model_spotify.most_similar(positive = list1,topn = 50)
    print ("Playlist based on your list is as follows")
    for i in similar[:]:
        print (track_map_comp_dict[i[0]])

In [63]:
create_play_list(['wonderwall','paradise','yellow','let her go','fireflies'])

Searching for songs similar to : ['wonderwall', 'paradise', 'yellow', 'let her go', 'fireflies']
Playlist based on your list is as follows
world spins madly on
you and your heart
wouldn't it be nice - 1999 - remaster
where the streets have no name - unplugged
wonderwall - remastered
yellow - live
wherever you will go - acoustic
won't go home without you
you are the best thing
you and i both
we can't stop (feat bea miller)
why does it always rain on me?
you found me
over my head (cable car) - acoustic [live]
wild world - acoustic version
you get what you give
you could be happy
white houses
wonderful tonight
wordplay
where i stood
people help the people
why do you let me stay here?
ya no se que hacer conmigo
wherever you will go
worn me down
where we gonna go from here
you always make me smile
wrong impression
ya hey
wrapped up
where you belong - from the fifty shades of grey" soundtrack"
wildfire
who knew
when we dig for gold in the usa
viva la vida - live
women of the world (go on str

C:\Users\ash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


In [68]:
create_play_list(['enter sandman','fade to black','kashmir'])

Searching for songs similar to : ['enter sandman', 'fade to black', 'kashmir']
Playlist based on your list is as follows
fear of the dark - 1998 remastered version
eye of the tiger
for whom the bell tolls
fear of the dark
fairies wear boots
feel good inc
du hast
even flow
fuel
entre dos tierras
ett slag färgat rött
everlong
fade to black - instrumental version
fortunate son
estranged
dream on
highway to hell
faint
down with the sickness
f**k the system
free bird
fuel (live)
fiesta pagana
en livstid i krig
electric crown
eruption
engel
fast as a shark
dr feelgood
epic
fighting the world
fake it
fat bottomed girls - 2011 remaster
heart-shaped box
firth of fifth - 2008 digital remaster
good times bad times
duality
guerilla radio
for those about to rock (we salute you)
flight of icarus
fallen - edit
entre dos tierras - live
fear of the dark - live
don't cry (original)
fallen leaves
entre dos aguas
enter the maze
even flow (remastered 2008)
feuer frei
expendable youth


C:\Users\ash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


In [71]:
create_play_list(['hey you','time','hypnotised','fix you'])

Searching for songs similar to : ['hey you', 'time', 'hypnotised', 'fix you']
Playlist based on your list is as follows
hänsyn
holy dread!
hon har ett sätt - 1998 digital remaster
high speed
highway of endless dreams
i am a man of constant sorrow - o brother, where art thou? soundtrack/with band
holiday
gap
human
i always was your girl
hyperspace
higher love
hold on
hey you - live
high
flakes
heaven
highly strung
för dig
hyenas
green eyes
hoodoo
hypodermic
here i stand and face the rain
historia de una hache que no quería ser muda
high roller
i don't like mondays
guilty - 2001 remastered version
high hopes - live
gambler
hip hip chin chin - yaziko club mix
heartbeats
hores bruixes
here lies love
god put a smile upon your face
hometown glory (album)
how'd you like that
home
good vibes - rub club mix
friday i'm in love
goodbye
hide and seek
hallelujah - radio edit
hilli
heat of the moment
hunting high and low - live
hame baham
heroes
hey there delilah
give it away


C:\Users\ash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


In [69]:
similar_songs('kashmir')

Searching for songs similar to : kashmir
Similar songs are as follow
kashmir - live: o2 arena, london - december 10, 2007
immigrant song
keep talking - 2011 remastered version
karma police
joker and the thief
kickstart my heart - international
killing in the name
key to the highway/trouble in mind - rough mix
kindred spirits
la grange


C:\Users\ash\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


### Highlighting the Problems with current datasets and next steps for further 

1. Training Data is not clean and has lot of similar songs with different names. We could try to restrict the version of songs to 1 or 2 max based on frequency for example
   SOngs :  kashmir , kashmir - live: o2 arena, london - december 10, 2007
   
2. Songs with similar names can be of different taste based on the artist names. We should create vocab by combining strings of tracknames with the artist names

        